In [1]:
%%capture
!pip3 install faiss-gpu-cu12
!pip3 install pgvector
!pip3 install kagglehub

In [2]:
%%capture
!git clone https://github.com/HackerBschor/SofteningQueryEvaluation
%cd SofteningQueryEvaluation

In [3]:
from huggingface_hub import notebook_login
notebook_login()

# Evaluate Similarity Grouping


In [4]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from models import ModelMgr
from models.embedding.SentenceTransformer import SentenceTransformerEmbeddingModel
from models.semantic_validation import LLaMAValidationModel

from db.operators import Dummy, Select
from db.criteria import SoftEqual
from db.structure import Column, Constant
import kagglehub

from evaluation.util import calculate_metrics, calc_bleu

import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
# Models
# meta-llama/Llama-3.2-3B-Instruct
# meta-llama/Meta-Llama-3-8B-Instruct

stem = SentenceTransformerEmbeddingModel(ModelMgr())
lsv = LLaMAValidationModel(ModelMgr(), model_path = "meta-llama/Llama-3.2-3B-Instruct")
# lsv = DeepSeekValidationModel(m)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [6]:
ZERO_SHOT_SYSTEM_PROMPT = "You are a validator. Respond with \"no\" and \"yes\" only!"
ZERO_SHOT_PROMPTING_TEMPLATE = 'Does "{}" describes "{}"'

random_state = 420
max_number_categories = 10
max_number_per_categories = 10

target = "category_1"

In [7]:
path = kagglehub.dataset_download("lakritidis/product-classification-and-categorization")

df = pd.read_csv(f"{path}/pricerunner_aggregate.csv", header=None, index_col=0)
df.drop(columns=[2, 3, 5], inplace=True)
df = df.drop_duplicates()
df.columns = ["product", "category_1", "category_2"]
df.head()

100%|██████████| 22.8M/22.8M [00:02<00:00, 9.42MB/s]

Extracting files...


,product,category_1,category_2
0,,,
1,apple iphone 8 plus 64gb silver,Apple iPhone 8 Plus 64GB,Mobile Phones
2,apple iphone 8 plus 64 gb spacegrau,Apple iPhone 8 Plus 64GB,Mobile Phones
3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,Apple iPhone 8 Plus 64GB,Mobile Phones
4,apple iphone 8 plus 64gb space grey,Apple iPhone 8 Plus 64GB,Mobile Phones
5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,Apple iPhone 8 Plus 64GB,Mobile Phones


In [8]:
categories = df[target].value_counts()
categories = pd.Series(categories[categories >= 10].index)
categories = list(categories.sample(min(max_number_categories, len(categories)), random_state=random_state))

print(len(categories))
print(", ".join(categories)[:100], "...")

10
Samsung RF24HSESBSR Stainless Steel, Panasonic TX-65FX750B, Samsung UE55NU7400, Samsung Galaxy S9+ 6 ...


In [9]:
df_reduced = pd.concat([df[df[target] == category].sample(max_number_per_categories, random_state=random_state) for category in categories])

gt = {(x["product"], x[target]) for key, x in df_reduced.iterrows()}
print(str(gt)[0:100], "...", len(gt))

{('canon eos 2000d slr camera ef s 18 55 mm is ii lens kit', 'Canon EOS 2000D + 18-55mm IS II'), ('l ... 100


In [10]:
overall_result = {}

def evaluate(method, threshold, system_prompt=ZERO_SHOT_SYSTEM_PROMPT, prompt_template=ZERO_SHOT_PROMPTING_TEMPLATE):
    pred = []
    runtimes = []
    for category in tqdm(categories):
        d = Dummy("products", ["product"], [(x[0], ) for x in gt])
        s = Select(d, SoftEqual(Column("product"),Constant(category), method=method, em=stem, sv=lsv, threshold=threshold, zfs_system_prompt=system_prompt, zfs_prompt_template = prompt_template))

        tic = time.time()
        result = s.open().fetch_all()
        toc = time.time()

        pred.extend([(x["product"], category) for x in result])
        runtimes.append(toc - tic)

    scores = calculate_metrics(gt, set(pred), np.mean(runtimes))

    print(method, threshold, scores["F1 Score"])

    return scores, pred

In [11]:
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("threshold", thresh)
    overall_result[("threshold", thresh)] = res

100%|██████████| 10/10 [00:13<00:00,  1.36s/it]


threshold 0.1 0.199203187250996


100%|██████████| 10/10 [00:12<00:00,  1.25s/it]


threshold 0.2 0.24813895781637715


100%|██████████| 10/10 [00:12<00:00,  1.26s/it]


...

threshold 0.8 0.38095238095238093


100%|██████████| 10/10 [00:12<00:00,  1.26s/it]


threshold 0.9 0.13084112149532712


100%|██████████| 10/10 [00:12<00:00,  1.26s/it]

threshold 1.0 0.019801980198019802


In [12]:
res, pred = evaluate("zero-few-shot", None)
overall_result[("zero-few-shot", None)] = res

100%|██████████| 10/10 [00:47<00:00,  4.76s/it]

zero-few-shot None 0.6394557823129251


In [13]:
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("both", thresh)
    overall_result[("both", thresh)] = res
    if res["Recall"] == 0.0:
        break

100%|██████████| 10/10 [00:55<00:00,  5.51s/it]


both 0.1 0.6394557823129251


100%|██████████| 10/10 [00:45<00:00,  4.53s/it]


both 0.2 0.6394557823129251


100%|██████████| 10/10 [00:34<00:00,  3.50s/it]


...

both 0.8 0.2608695652173913


100%|██████████| 10/10 [00:12<00:00,  1.30s/it]


both 0.9 0.09523809523809523


100%|██████████| 10/10 [00:12<00:00,  1.26s/it]

both 1.0 0


In [14]:
for key in tqdm(overall_result):
    scores_bleu = calc_bleu(gt, overall_result[key]["pred"])
    for score_bleu in scores_bleu:
        overall_result[key][score_bleu] = scores_bleu[score_bleu]

100%|██████████| 21/21 [01:09<00:00,  3.32s/it]


In [15]:
df_results = pd.DataFrame([{"method": k[0], "threshold": k[1]} | v for k,v in overall_result.items()])
df_results

,method,threshold,Precision,Recall,F1 Score,tp,fn,fp,runtime,pred,bleu1,bleu2,bleu3,bleu4
0,threshold,0.1,0.110619,1.00,0.199203,100,0,804,1.356066,{(lg oled65c8pla 65 4k ultra hd hdr oled smart...,1.000000,1.000000,1.000000,1.000000
1,threshold,0.2,0.141643,1.00,0.248139,100,0,606,1.252160,{(lg oled65c8pla 65 4k ultra hd hdr oled smart...,1.000000,1.000000,1.000000,1.000000
2,threshold,0.3,0.202899,0.98,0.336192,98,2,385,1.255596,{(lg oled65c8pla 65 4k ultra hd hdr oled smart...,0.993150,0.992771,0.992306,0.991705
3,threshold,0.4,0.286982,0.97,0.442922,97,3,241,1.263670,{(lg oled65c8pla 65 4k ultra hd hdr oled smart...,0.990209,0.988328,0.986259,0.984770
4,threshold,0.5,0.453659,0.93,0.609836,93,7,112,1.258244,"{(samsung ue43nu7400/ue43nu7400, LG OLED65C8PL...",0.977597,0.970569,0.964910,0.960802
5,threshold,0.6,0.788462,0.82,0.803922,82,18,22,1.263840,"{(samsung ue55nu7400/ue55nu7400, Samsung UE43N...",0.924741,0.899740,0.880435,0.865132
6,threshold,0.7,0.881356,0.52,0.654088,52,48,7,1.263317,"{(samsung ue55nu7400/ue55nu7400, Samsung UE43N...",0.814209,0.761994,0.721625,0.678708
7,threshold,0.8,0.923077,0.24,0.380952,24,76,2,1.260020,"{(samsung ue55nu7400/ue55nu7400, Samsung UE43N...",0.608909,0.524225,0.465552,0.406563
8,threshold,0.9,1.000000,0.07,0.130841,7,93,0,1.258780,"{(lg oled65c8pla, LG OLED65C8PLA), (canon eos ...",0.378349,0.306100,0.244643,0.192010
9,threshold,1.0,1.000000,0.01,0.019802,1,99,0,1.262514,"{(lg oled65c8pla, LG OLED65C8PLA)}",0.114182,0.059267,0.044678,0.033632


In [16]:
df_results.to_pickle("Products_fine_mpnetBaseV2_LLama3B.pkl")
df_results.drop(columns=["pred"]).to_csv("Products_fine_mpnetBaseV2_LLama3B.csv")